<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 14:16:49
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.33 C
-------------------
Today PnL: -57.56 K (-0.43%)
Current PnL: -21.96 L (-14.93%)
CY Booked + Current PnL: -13.69 L (-9.31%)
-------------------
Total profit:  2.09 L
Total loss:  -24.05 L
-------------------
Total Booked + Current PnL: 13.29 L (11.07%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.22%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 80.21 L (60.35%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.61,H-SC,13.82,164906.0,38706.0,7470.0,7.13,30.67,4.53,36.59,156.0,5.18,1.27,47.03,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.64,107864.0,3552.0,13095.0,-1.12,3.41,12.14,15.96,10.0,0.27,0.83,20.88,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.94,213776.0,12296.0,14922.0,0.19,6.10,6.98,13.51,62.0,0.82,1.65,19.47,XY25,NTT,BANKS
82,TTKPRESTIG,769.29,770.00,-9.13,H-SC,11.15,83028.0,-17749.0,17843.0,-0.04,-17.61,21.49,0.09,145.0,-0.99,0.64,9.08,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2247.15,H-SC,7.07,63146.0,-17944.0,17946.0,-0.01,-22.13,28.42,0.00,157.0,-1.00,0.49,27.98,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.94,204198.0,-302.0,41718.0,-0.03,-0.15,20.43,20.25,9.0,-0.01,1.58,8.57,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.67,H-LC,2.46,169551.0,-31835.0,119449.0,-0.52,-15.81,70.45,43.51,19.0,-0.27,1.31,18.02,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.24,H-LC,3.53,180558.0,-19752.0,79734.0,-0.44,-9.86,44.16,29.94,24.0,-0.25,1.40,24.74,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-11.03,H-LC,3.65,184582.0,-15924.0,64696.0,-0.42,-7.94,35.05,24.33,30.0,-0.25,1.43,6.44,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-59.41,H-LC,3.85,128677.0,-136749.0,193659.0,-1.57,-51.52,150.50,21.44,26.0,-0.71,0.99,3.40,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.61,H-SC,13.82,164906.0,38706.0,7470.0,7.13,30.67,4.53,36.59,156.0,5.18,1.27,47.03,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.0,-51.71,L-SC,22.05,70790.0,-42759.0,82810.0,4.03,-37.66,116.98,35.27,267.0,-0.52,0.55,73.26,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119475.0,-23004.0,23620.0,2.34,-16.15,19.77,0.44,163.0,-0.97,0.92,41.01,OX40N,NTT,BANKS
86,VAIBHAVGBL,350.21,521.0,43.80,H-SC,7.86,96057.0,-55934.0,130061.0,1.44,-36.80,135.40,48.77,127.0,-0.43,0.74,11.03,XR,NTT,JEWELLERY
8,AWL,325.39,485.0,-69.74,H-MC,5.25,209715.0,-58732.0,190400.0,1.40,-21.88,90.79,49.05,82.0,-0.31,1.62,7.83,XY24,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LAOPALA,369.40,464.00,115.96,H-SC,3.49,68550.0,-32296.0,58124.0,-4.98,-32.03,84.79,25.61,141.0,-0.56,0.53,27.09,AR,NTT,CERAMICS
80,TITAGARH,1097.23,1548.00,-13.27,H-SC,6.15,142380.0,-49635.0,128526.0,-4.73,-25.85,90.27,41.08,158.0,-0.39,1.10,20.51,XY24,NTT,ENGINEERING
27,DIXON,14367.00,18940.15,-29.98,H-MC,6.55,128896.0,13960.0,22621.0,-3.31,12.15,17.55,31.83,85.0,0.62,1.00,40.72,X40N,ATH,IT
17,BSOFT,628.85,836.99,-22.90,H-SC,13.75,64838.0,-45211.0,81638.0,-3.00,-41.08,125.91,33.10,129.0,-0.55,0.50,7.74,XR,ATH,IT
61,RELAXO,902.64,1176.00,-37.64,H-SC,12.05,56384.0,-59154.0,94144.0,-2.19,-51.20,166.97,30.28,134.0,-0.63,0.44,10.15,X40N,NTT,FOOTWEAR


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,124.33,H-SC,3.91,147188.0,-27371.0,27406.0,-1.91,-15.68,18.62,0.02,135.0,-1.00,1.14,27.63,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119475.0,-23004.0,23620.0,2.34,-16.15,19.77,0.44,163.0,-0.97,0.92,41.01,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-9.13,H-SC,11.15,83028.0,-17749.0,17843.0,-0.04,-17.61,21.49,0.09,145.0,-0.99,0.64,9.08,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-22.52,H-MC,2.14,81375.0,-21277.0,21271.0,-0.90,-20.73,26.14,-0.00,101.0,-1.00,0.63,23.70,OX40N,NTT,IT
20,CERA,7989.07,8422.0,-21.48,H-SC,3.00,112338.0,-31465.0,39262.0,-0.04,-21.88,34.95,5.42,155.0,-0.80,0.87,23.97,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,15.37,237167.0,-58767.0,152688.0,-0.21,-19.86,64.38,31.73,1.0,-0.38,1.83,1.25,X40,BTT,IT
88,VBL,492.64,671.64,-11.37,H-LC,11.01,269081.0,4041.0,92268.0,-0.51,1.52,34.29,36.33,3.0,0.04,2.08,15.03,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.32,H-LC,9.50,255218.0,-6383.0,152008.0,-0.77,-2.44,59.56,55.67,4.0,-0.04,1.97,3.52,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.63,H-LC,9.43,197730.0,-18789.0,111698.0,-0.76,-8.68,56.49,42.91,5.0,-0.17,1.53,3.55,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,1.41,H-LC,9.89,180084.0,-10562.0,57069.0,0.04,-5.54,31.69,24.40,8.0,-0.19,1.39,9.51,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.61,H-SC,13.82,164906.0,38706.0,7470.0,7.13,30.67,4.53,36.59,156.0,5.18,1.27,47.03,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.61,H-SC,13.82,164906.0,38706.0,7470.0,7.13,30.67,4.53,36.59,156.0,5.18,1.27,47.03,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.64,25.10,H-SC,13.19,127852.0,2555.0,56523.0,-1.05,2.04,44.21,47.15,144.0,0.05,0.99,59.95,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,0.48,H-MC,6.06,161640.0,10727.0,46585.0,-0.91,7.11,28.82,37.97,90.0,0.23,1.25,34.21,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-3.59,H-SC,7.94,180700.0,16030.0,51933.0,-1.12,9.73,28.74,41.28,119.0,0.31,1.40,18.49,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.92,H-SC,2.39,105596.0,3592.0,40338.0,-0.96,3.52,38.20,43.06,120.0,0.09,0.82,21.24,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.48,H-MC,6.06,161640.0,10727.0,46585.0,-0.91,7.11,28.82,37.97,90.0,0.23,1.25,34.21,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-47.89,H-SC,13.34,135659.0,12323.0,121442.0,-0.02,9.99,89.52,108.45,118.0,0.10,1.05,34.58,AR,ATH,MISC
50,KPIGREEN,497.21,731.64,25.10,H-SC,13.19,127852.0,2555.0,56523.0,-1.05,2.04,44.21,47.15,144.0,0.05,0.99,59.95,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-43.35,M-SC,4.89,95198.0,3700.0,75054.0,-1.96,4.04,78.84,86.07,211.0,0.05,0.74,37.71,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-12.85,M-LC,6.49,149222.0,-1723.0,111185.0,-0.68,-1.14,74.51,72.51,55.0,-0.02,1.15,4.81,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.05,70790.0,-42759.0,82810.0,4.03,-37.66,116.98,35.27,267.0,-0.52,0.55,73.26,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.83,H-SC,17.92,89483.0,-11480.0,106011.0,-1.29,-11.37,118.47,93.63,149.0,-0.11,0.69,33.28,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.05,91710.0,-6270.0,28137.0,0.91,-6.40,30.68,22.32,151.0,-0.22,0.71,33.01,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.64,107864.0,3552.0,13095.0,-1.12,3.41,12.14,15.96,10.0,0.27,0.83,20.88,X5K,ATH,METALS
83,UJJIVANSFB,52.77,53.00,37.69,H-SC,20.69,119475.0,-23004.0,23620.0,2.34,-16.15,19.77,0.44,163.0,-0.97,0.92,41.01,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.61,H-SC,13.82,164906.0,38706.0,7470.0,7.13,30.67,4.53,36.59,156.0,5.18,1.27,47.03,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.68,H-SC,8.05,217178.0,24158.0,23890.0,-0.60,12.52,11.00,24.89,128.0,1.01,1.68,38.78,X40N,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,944.66,M-SC,8.16,142093.0,-39050.0,84517.0,-0.33,-21.56,59.48,25.10,233.0,-0.46,1.10,13.87,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.05,70790.0,-42759.0,82810.0,4.03,-37.66,116.98,35.27,267.0,-0.52,0.55,73.26,XR,NTT,HOTELS
88,VBL,492.64,671.64,-11.37,H-LC,11.01,269081.0,4041.0,92268.0,-0.51,1.52,34.29,36.33,3.0,0.04,2.08,15.03,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,41.30
2,50,71.37


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.05
LC    36.56
MC    22.39
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.25
X40N     15.39
X40      13.48
XY25     10.81
XR        9.05
AR        7.75
OX40N     7.73
X5K       2.25
MH        1.76
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.29
H-SC    27.62
H-MC    19.41
M-SC    11.96
M-LC     7.18
M-MC     2.63
L-SC     1.47
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.37,-10.80,46.42
IT,12.06,-19.50,77.55
FINANCE,8.39,-20.24,63.86
BANKS,7.63,-15.24,62.26
PAINTS,5.80,-16.44,39.38
HEALTHCARE,5.26,3.51,20.85
MISC,5.20,-21.58,92.65
ELECTRICAL,4.77,-14.02,63.39
AUTO,4.46,-16.96,67.84


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2851251.0,23
XR,1120819.0,13
AR,1062471.0,9
X40,827402.0,10
XY25,649500.0,8
X40N,619524.0,12
OX40N,378013.0,10
SR,196833.0,2
X5K,147424.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2748594.0,30
H-LC,1494156.0,20
M-SC,1423485.0,15
H-MC,1194033.0,15
M-LC,448312.0,5
M-MC,337692.0,2
L-SC,271537.0,3
L-MC,57546.0,1
L-LC,45795.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947824.0      7
M-SC       XY24      863266.0      7
H-SC       AR        621342.0      4
H-LC       X40       548984.0      6
H-SC       XR        524320.0      6
H-MC       XY24      469123.0      4
M-MC       XY24      337692.0      2
H-LC       X40N      320050.0      6
           XY25      252875.0      3
           AR        233948.0      2
H-MC       XY25      212459.0      2
H-SC       X40N      210305.0      4
H-MC       X40       202387.0      3
H-SC       SR        196833.0      2
           OX40N     191447.0      6
M-LC       XY24      187876.0      2
L-SC       XR        181585.0      2
M-SC       AR        160596.0      2
M-LC       X5K       134329.0      1
M-SC       XR        124800.0      2
           XY25      123449.0      1
H-MC       XR        121383.0      1
M-LC       XR        111185.0      1
L-SC       OX40N      89952.0      1
H-MC       X40N       89169.0      2
H-LC       X200       79734.0      1
M-SC       X40        76031.0      1
           OX40N      75343.0      2
L-MC       XR         57546.0      1
H-SC       MH         56523.0      1
H-MC       AR         46585.0      1
L-LC       XY25       45795.0      1
H-LC       XY24       45470.0      1
H-MC       MH         31656.0      1
           OX40N      21271.0      1
M-LC       XY25       14922.0      1
H-LC       X5K        13095.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
